In [1]:
import sys
from unipath import Path

# append codelib to sys path ~ will only work if this jupyter notebook path is sys.path[0]
p = Path(sys.path[0])
codelib_path = str(p.parent + Path('\\codelib'))
sys.path.insert(0, codelib_path)

# import codelib
from helpers import *
from mean_variance import *
import enhanced_portfolio_optimization as epo

# other modules
import numpy as np
import matplotlib.pyplot as plt

# Import data

In [5]:
# setting dates
start = "2000-01-01"
end   = "2022-01-01"
port  = "5_Industry_Portfolios"

# pulling data
df = pdr.famafrench.FamaFrenchReader(port, start, end).read()[0]
df = df.replace(-99.99,0) # replace nan values
df = df / 100 # changing format to decimals

# risk free rate
rf = 0.0001

# returns in array
ret = df.to_numpy()

print("Number of assets:", ret.shape[1])
print("Number of observations:", ret.shape[0])

Number of assets: 5
Number of observations: 265


# Equation for anchored EPO

The simple equation for the aEPO is as follows.

$$
\text{EPO}^a (w) = \Sigma_w^{-1} ( [1-w] \frac{\sqrt{a^\intercal \tilde{\Sigma} a}}{\sqrt{s^\intercal \Sigma_w^{-1} \tilde{\Sigma} \Sigma_w^{-1} s}} s + wVa )  
$$

* $w$: Shrinkage parameter ~ $w \in [0,1]$
* $\Sigma_w^{-1}$: Inversed shrunk variance-covariance matrix.
* $a$: Vector of weights in anchor portfolio.
* $\tilde{\Sigma}$: Enhanced variance-covariance matrix.
* $s$: Vector of signals about expected excess return.
* $V$: Diagonal matrix of variances.

**Notes**

There is something about $s$ needing to be <u>scaled</u> to the excess return (p. 16). So that $s_i = 0.02$ means $E(r_i) = 0.02$. I think this is because $s$ functions as $\mu$ in this equation, thus, $s$ is simply $\mu$ with incorporated signals. Therefore, it must be scaled to match the other inputs.


## Full period example

Here, I will simply use the full input as an example for calculating the weights. Thus, no validations or train-test-split.

**Anchor portfolio** <br>
The anchor portfolio will simply be the equal weighted portfolio.

$$
a = \frac{1}{N}
$$

**Signals about expected returns** <br>
For simplicity there is going to be no signals or conditional estimation of expected returns. Signals will be a simple mean of returns.

In [6]:
def shrunk_variance_covariance(return_series: np.ndarray, shrinkage: float):
    
    # variances
    var = np.var(return_series, axis=0)
    
    # correlation matrix
    corr = np.corrcoef(return_series.T)
    
    # shrunk correlation matrix
    shr_corr = corr * (1 - shrinkage) + np.identity(len(return_series[0])) * shrinkage
    
    # shrunk variance covariance
    cov = np.outer(var, var) * shr_corr
    
    return cov

In [22]:
# shrinkage parameter ~ from paper
w = 0.75

# shrunk variance covariance matrix
shr_Sigma = shrunk_variance_covariance(ret, w)

# anchor portfolio
weight = 1 / len(shr_Sigma)
a = np.linspace(weight, weight, len(shr_Sigma))

# enhanced variance-covariance matrix
enh_Sigma = np.cov(ret.T)

# signals
s = np.mean(ret, axis=0)

# diagonal matrix of variances
V = np.diag(np.var(ret, axis=0))

In [35]:
def anchored_epo(shrinkage: float, 
                 shr_Sigma: np.ndarray, 
                 anchor: np.ndarray, 
                 enh_Sigma: np.ndarray, 
                 signals: np.ndarray,
                 diag_var: np.ndarray):
    
    # inputs
    inv_shr_Sigma = np.linalg.inv(shr_Sigma)
    
    nominator = np.sqrt(anchor.T * enh_Sigma * anchor)
    denominator = np.sqrt(signals.T * inv_shr_Sigma * enh_Sigma * inv_shr_Sigma * signals)
    
    a_epo = inv_shr_Sigma * ((1 - shrinkage) * nominator/denominator * signals + shrinkage * diag_var * anchor)

    return nominator


In [70]:
anchored_epo(w, shr_Sigma, a, enh_Sigma, s, V)

array([[ 1.01681611e+02, -5.00000000e-02, -5.00000000e-02,
        -5.00000000e-02, -5.00000000e-02],
       [-5.00000000e-02,  7.75811958e+01, -5.00000000e-02,
        -5.00000000e-02, -5.00000000e-02],
       [-5.00000000e-02, -5.00000000e-02,  4.22004053e+01,
        -5.00000000e-02, -5.00000000e-02],
       [-5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
         9.56812908e+01, -5.00000000e-02],
       [-5.00000000e-02, -5.00000000e-02, -5.00000000e-02,
        -5.00000000e-02,  5.87752709e+01]])